In [1]:
from transformers import Qwen2ForCausalLM, Qwen2Model, AutoTokenizer
import torch

torch.set_grad_enabled(False)
model_name = "Qwen/Qwen2.5-0.5B-Instruct"
model = Qwen2ForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map={"": 0},
)
model = model.eval()

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [2]:
model.get_input_embeddings().weight.dtype, model.dtype

(torch.bfloat16, torch.bfloat16)

In [3]:
device = "cuda"
tokenizer = AutoTokenizer.from_pretrained(model_name)

prompt = "Give me a short introduction to large language model..1"
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt},
]
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(model_inputs.input_ids, max_new_tokens=512)
generated_ids = [
    output_ids[len(input_ids) :]
    for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
response

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


'A large language model is a type of artificial intelligence that can generate human-like text based on input data, such as natural language sentences or questions. These models are designed to mimic the complexity and creativity of human language, allowing them to produce coherent and meaningful responses that can be used in various applications, including speech recognition, chatbots, and virtual assistants. Large language models have been trained on vast amounts of text data, enabling them to understand and respond to complex queries and ideas more accurately than traditional AI systems.'

In [6]:
model_output_1 = model(
    **model_inputs,
    output_hidden_states=True,
)
model_output_1.hidden_states[-1].shape, model_output_1.hidden_states[-1].dtype

(torch.Size([1, 30, 896]), torch.bfloat16)

In [7]:
model_output_2 = model(
    inputs_embeds=torch.cat(
        [
            model_output_1.hidden_states[-1],
            model_output_1.hidden_states[-1],
        ],
        dim=0,
    ),
    attention_mask=model_inputs["attention_mask"],
    output_hidden_states=True,
)
model_output_2.hidden_states[-1].shape

torch.Size([2, 30, 896])

In [8]:
model_output_1.hidden_states[-1].shape

torch.Size([1, 30, 896])

In [9]:
model_output_1.hidden_states[-1].reshape(10, 3, -1).shape

torch.Size([10, 3, 896])

In [11]:
model_output_2 = model(
    inputs_embeds=torch.cat(
        [
            model_output_1.hidden_states[-1],
            model_output_1.hidden_states[-1],
        ],
        dim=0,
    ),
    attention_mask=model_inputs["attention_mask"],
    output_hidden_states=True,
)
model_output_2.hidden_states[-1].shape, model_output_2.hidden_states[-1].dtype

(torch.Size([2, 30, 896]), torch.bfloat16)

In [12]:
model_output_3 = model(
    inputs_embeds=model_output_1.hidden_states[-1].reshape(10, 3, -1),
    # attention_mask=model_inputs["attention_mask"],
    output_hidden_states=True,
)
model_output_3.hidden_states[-1].shape

torch.Size([10, 3, 896])

In [13]:
model_output_1.hidden_states[-1].reshape(1, 10, 3, -1).shape

torch.Size([1, 10, 3, 896])

In [12]:
# model(
#     inputs_embeds=model_output_1.hidden_states[-1].reshape(1, 10, 3, -1),
#     attention_mask=model_inputs["attention_mask"],
#     output_hidden_states=True,
# )

In [36]:
class Qwen2ModelEmbedPooler(Qwen2ForCausalLM):
    def __init__(self, config):
        super().__init__(config)
        self.model = Qwen2Model(config).cuda()
        self.post_init()

    def forward(self, input_embeds, attention_mask, window_size=3):
        # разбиваем входящие эмбединги на бакеты и усредняем их
        sum_mask = attention_mask.reshape(
            attention_mask.shape[0],
            window_size,
            attention_mask.shape[1] // window_size,
            -1,
        ).sum(1)
        embeds_sum = input_embeds.reshape(
            attention_mask.shape[0],
            window_size,
            attention_mask.shape[1] // window_size,
            -1,
        ).sum(1)
        input_embeds = embeds_sum / sum_mask
        input_embeds = self.model(
            inputs_embeds=input_embeds,
            output_hidden_states=True,
        )
        return input_embeds


embed_pooler = Qwen2ModelEmbedPooler.from_pretrained("Qwen/Qwen2.5-0.5B")
result = embed_pooler(
    # model_output_1.hidden_states[-1],
    torch.cat(
        [
            model_output_1.hidden_states[-1],
            model_output_1.hidden_states[-1],
        ],
        dim=0,
    ),
    torch.cat(
        [
            model_inputs["attention_mask"],
            model_inputs["attention_mask"],
        ],
        dim=0,
    ),
)
result[0].shape

torch.Size([2, 10, 896])

In [15]:
result.last_hidden_state.shape

torch.Size([2, 10, 896])

In [9]:
torch.tensor([1, 2, 3, 4, 5, 6]).reshape(3, -1)

tensor([[1, 2],
        [3, 4],
        [5, 6]])

In [ ]:
from typing import Callable, List, Optional, Tuple, Union
from transformers.cache_utils import (
    Cache,
    DynamicCache,
    SlidingWindowCache,
    StaticCache,
)
from transformers.processing_utils import Unpack
from transformers.modeling_flash_attention_utils import FlashAttentionKwargs
from transformers.utils import LossKwargs
from transformers.modeling_outputs import (
    BaseModelOutputWithPast,
    CausalLMOutputWithPast,
    QuestionAnsweringModelOutput,
    SequenceClassifierOutputWithPast,
    TokenClassifierOutput,
)


class KwargsForCausalLM(FlashAttentionKwargs, LossKwargs): ...


class Qwen2ForCausalEmbedModeling(Qwen2ForCausalLM):
    def __init__(self, config):
        super().__init__(config)
        self.model = Qwen2Model(config)
        self.vocab_size = config.vocab_size
        self.lm_head = torch.nn.Linear(
            config.hidden_size,
            config.vocab_size,
            bias=False,
        )
        self.embed_pooler = Qwen2ModelEmbedPooler.from_pretrained("Qwen/Qwen2.5-1.5B")

        # Initialize weights and apply final processing
        self.post_init()

    def forward(
        self,
        input_ids: torch.LongTensor = None,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        past_key_values: Optional[Union[Cache, List[torch.FloatTensor]]] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        cache_position: Optional[torch.LongTensor] = None,
        logits_to_keep: Union[int, torch.Tensor] = 0,
        # pooled_mask: Optional[torch.Tensor] = None,
        **kwargs: Unpack[KwargsForCausalLM],
    ) -> Union[Tuple, CausalLMOutputWithPast]:
        output_attentions = (
            output_attentions
            if output_attentions is not None
            else self.config.output_attentions
        )
        output_hidden_states = (
            output_hidden_states
            if output_hidden_states is not None
            else self.config.output_hidden_states
        )
        return_dict = (
            return_dict if return_dict is not None else self.config.use_return_dict
        )

        inputs_embeds_tokens = self.model.embed_tokens(input_ids)
        # if pixel_values is not None:
        #     pixel_values = pixel_values.type(self.visual.get_dtype())
        #     image_embeds = self.visual(pixel_values, grid_thw=image_grid_thw)
        #     n_image_tokens = (input_ids == self.config.image_token_id).sum().item()
        #     n_image_features = image_embeds.shape[0]
        #     if n_image_tokens != n_image_features:
        #         raise ValueError(
        #             f"Image features and image tokens do not match: tokens: {n_image_tokens}, features {n_image_features}"
        #         )
        #     image_mask = (
        #         (input_ids == self.config.image_token_id)
        #         .unsqueeze(-1)
        #         .expand_as(inputs_embeds)
        #         .to(inputs_embeds.device)
        #     )
        #     image_embeds = image_embeds.to(inputs_embeds.device, inputs_embeds.dtype)
        #     inputs_embeds = inputs_embeds.masked_scatter(image_mask, image_embeds)
        window_size = torch.tensor(
            [
                [4],
                [3],
            ]
        ).cuda()
        tokens_amount = torch.tensor(
            [
                [2],
                [4],
            ]
        ).cuda()
        lengths = window_size * tokens_amount
        # max_len = lengths.max()
        max_len = inputs_embeds_tokens.shape[1]
        batch_size = window_size.shape[0]
        pooled_mask = (
            torch.arange(max_len, device=device)
            .unsqueeze(0)
            .expand(batch_size, max_len)
        )
        pooled_mask = (lengths >= pooled_mask).to(torch.long)
        pooled_embeds = inputs_embeds * pooled_mask.to(inputs_embeds.dtype)
        pooled_embeds = self.embed_pooler(pooled_embeds, pooled_mask)
        embed_mask = (
            (input_ids == self.config.image_token_id)
            .unsqueeze(-1)
            .expand_as(inputs_embeds)
            .to(inputs_embeds.device)
        )
        inputs_embeds = inputs_embeds.masked_scatter(embed_mask, pooled_embeds)

        # Из-за смешанной структуры, мы будем всегда подавать только эмбединги
        # Идея позаимствована из qwen2vl
        outputs = self.model(
            input_ids=None,
            attention_mask=attention_mask,
            position_ids=position_ids,
            past_key_values=past_key_values,
            inputs_embeds=inputs_embeds,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
            cache_position=cache_position,
            **kwargs,
        )

        hidden_states = outputs[0]
        # Only compute necessary logits, and do not upcast them to float if we are not computing the loss
        slice_indices = (
            slice(-logits_to_keep, None)
            if isinstance(logits_to_keep, int)
            else logits_to_keep
        )
        logits = self.lm_head(hidden_states[:, slice_indices, :])

        loss = None
        if labels is not None:
            loss = self.loss_function(
                logits=logits,
                labels=labels,
                vocab_size=self.config.vocab_size,
                **kwargs,
            )

        if not return_dict:
            output = (logits,) + outputs[1:]
            return (loss,) + output if loss is not None else output

        return CausalLMOutputWithPast(
            loss=loss,
            logits=logits,
            past_key_values=outputs.past_key_values,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

## Использование концепции qwen2vl для текстовых токенов и эмбедингов

In [14]:
text_example = "Deep learning, a subfield of machine learning, has revolutionized the landscape of artificial intelligence in recent years. From self-driving cars to personalized medicine, its applications are becoming increasingly pervasive. But what exactly is deep learning? And what makes it so powerful?"

tokenizer.encode(text_example)[:10]

[33464, 6832, 11, 264, 1186, 2566, 315, 5662, 6832, 11]

In [15]:
tokenizer.encode("<|image_pad|>")

[151655]

In [16]:
tokenizer.encode("text_example")

[1318, 39304]

### 1. Случай первый - мы сжимаем только текстовые токены (максимально упрощенный вариант)

In [17]:
window_size = 3
chunk_size = 4
new_tokens = []
original_tokens = tokenizer.encode(text_example)
new_tokens += tokenizer.encode("<|object_ref_start|>") * chunk_size
new_tokens += original_tokens[chunk_size * window_size :]
tokenizer.decode(new_tokens)

'<|object_ref_start|><|object_ref_start|><|object_ref_start|><|object_ref_start|>ized the landscape of artificial intelligence in recent years. From self-driving cars to personalized medicine, its applications are becoming increasingly pervasive. But what exactly is deep learning? And what makes it so powerful?'

In [18]:
tokenizer.decode(original_tokens)

'Deep learning, a subfield of machine learning, has revolutionized the landscape of artificial intelligence in recent years. From self-driving cars to personalized medicine, its applications are becoming increasingly pervasive. But what exactly is deep learning? And what makes it so powerful?'

In [21]:
original_tokens_torch = torch.tensor(
    [
        original_tokens,
        original_tokens,
    ],
    device="cuda",
)
new_tokens_torch = torch.tensor(
    [
        new_tokens,
        new_tokens,
    ],
    device="cuda",
)
# torch.Size([2, 51, 896])
original_embeds = model.get_input_embeddings()(original_tokens_torch)
# torch.Size([2, 43, 896]) 51 - 3*4 + 4
compressed_embeds_template = model.get_input_embeddings()(new_tokens_torch)
compressed_embeds_template.shape, compressed_embeds_template.dtype, original_embeds.dtype

(torch.Size([2, 43, 896]), torch.bfloat16, torch.bfloat16)

In [ ]:
class Qwen2ModelEmbedPoolerV2(Qwen2ForCausalLM):
    def __init__(self, config):
        super().__init__(config)
        self.model = Qwen2Model(config)
        self.post_init()

    def forward(self, input_embeds):
        # print(input_embeds.dtype)
        input_embeds = self.model(
            inputs_embeds=input_embeds,
            output_hidden_states=True,
        )[0]
        # print(input_embeds.dtype)
        input_embeds = input_embeds.sum(1) / torch.tensor(
            input_embeds.shape[1],
            device=input_embeds.device,
        )
        # print(input_embeds.dtype)
        input_embeds = input_embeds.unsqueeze(1)
        return input_embeds


embed_pooler_v2 = Qwen2ModelEmbedPoolerV2.from_pretrained(
    "Qwen/Qwen2.5-0.5B",
    torch_dtype=torch.bfloat16,
    device_map={"": 0},
)

# torch.Size([8, 3, 896])
compressed_embeds = original_embeds[:, : chunk_size * window_size].reshape(
    chunk_size * original_embeds.shape[0],
    window_size,
    -1,
)
compressed_embeds.shape
# torch.Size([8, 1, 896])
pooled_embeds = embed_pooler_v2(compressed_embeds)
# torch.Size([2, 4, 896])
pooled_embeds = pooled_embeds.reshape(
    original_embeds.shape[0],
    chunk_size,
    -1,
)
# torch.Size([2, 43, 896])
compressed_embeds_template[:, :chunk_size] = pooled_embeds
compressed_embeds_template.shape, compressed_embeds_template.dtype, pooled_embeds.dtype

(torch.Size([2, 43, 896]), torch.bfloat16, torch.bfloat16)

In [25]:
embed_pooler_v2.model.dtype

torch.bfloat16

In [26]:
labels = new_tokens_torch.clone()
text_token_id = tokenizer.encode("<|object_ref_start|>")[0]
labels[labels == text_token_id] = -100
labels

tensor([[ -100,  -100,  -100,  -100,  1506,   279, 18414,   315, 20443, 11229,
           304,  3213,  1635,    13,  5542,   656, 59711,  9331,   311, 34549,
         15712,    11,  1181,  8357,   525, 10454, 14756, 70767,    13,  1988,
          1128,  6896,   374,  5538,  6832,    30,  1597,  1128,  3643,   432,
           773,  7988,    30],
        [ -100,  -100,  -100,  -100,  1506,   279, 18414,   315, 20443, 11229,
           304,  3213,  1635,    13,  5542,   656, 59711,  9331,   311, 34549,
         15712,    11,  1181,  8357,   525, 10454, 14756, 70767,    13,  1988,
          1128,  6896,   374,  5538,  6832,    30,  1597,  1128,  3643,   432,
           773,  7988,    30]], device='cuda:0')

In [27]:
model_output_1 = model(
    inputs_embeds=compressed_embeds_template,
    labels=labels,
    output_hidden_states=True,
)
model_output_1.loss

tensor(8.7288, device='cuda:0')

### 2. Усложненная версия токенизации текста. Сжатые токены возникают между обычным тектом

Входная последовательность может быть 5 видов.

- когда у нас на входе только текст, который мы просто моделируем 
- когда на входе текст, но мы хотим сжать его некоторые части
- когда на входе текст и эмбединги с последнего слоя, где мы хотим сжать только части текста
- когда на входе текст и эмбединги с последнего слоя, где мы хотим сжать, часть текста и эмбедингов, совместно (токены переводим в эмбединги и сжимаем вместе с hidden states)
- когда на входе текст и эмбединги с последнего слоя, где мы хотим сжать, только эмбединги

Как мы решаем какие токены хотим сжать?
- никак. Просто говорим сожми с такого по такой.
- это норм только на этапе обучения, так как мы можем перебрать все комбинации.
- не норм на этапе инференса. например мы можем сжимать после каждых сгенеренных 10 токенов или 100, 1000. а какое окно контекста? 3, 10, 100? А что если менять стратегию. Сначала мы сжимали с окном 5 токенов, потом 20?
- Кажется что эти гиперпараметры можно найти простым перебором на валидации. Однако перебор стратегии уже не кажется таким очевидным. Напрашивается RL.


In [28]:
text_example = [
    "Deep learning, a subfield of machine learning, has revolutionized the landscape of artificial intelligence in recent years. From self-driving cars to personalized medicine, its applications are becoming increasingly pervasive. But what exactly is deep learning? And what makes it so powerful? ",
    """Before diving into the "deep" part, let's establish a foundation with the basics of artificial neural networks (ANNs). An ANN consists of interconnected nodes, called neurons, organized in layers. These layers typically include: These layers typically include: These layers typically include: networks (ANNs). An ANN consists of interconnected nodes, called neurons, organized in layers. These layers typically include: These layers typically include: These layers typically include: networks (ANNs). An ANN consists of interconnected nodes, called neurons, organized in layers. These layers typically include: These layers typically include: These layers typically include:""",
]

tokenizer.batch_encode_plus(
    text_example,
    padding=True,
)

{'input_ids': [[33464, 6832, 11, 264, 1186, 2566, 315, 5662, 6832, 11, 702, 13791, 1506, 279, 18414, 315, 20443, 11229, 304, 3213, 1635, 13, 5542, 656, 59711, 9331, 311, 34549, 15712, 11, 1181, 8357, 525, 10454, 14756, 70767, 13, 1988, 1128, 6896, 374, 5538, 6832, 30, 1597, 1128, 3643, 432, 773, 7988, 30, 220, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643], [10227, 42415, 1119, 279, 330, 32880, 1, 949, 11, 1077, 594, 5695, 264, 16266, 448, 279, 31774, 315, 20443, 29728, 14155, 320, 

In [24]:
torch.tensor(
    tokenizer.batch_encode_plus(
        text_example,
        padding=True,
    )["input_ids"]
).shape

torch.Size([2, 122])

In [29]:
tokenizer.decode(151643)

'<|endoftext|>'

In [ ]:
tokenizer

In [30]:
from pprint import pprint
from more_itertools import chunked
import numpy as np
import random

text_token_id = tokenizer.encode("<|object_ref_start|>")[0]
eos_token_id = tokenizer.encode("<|endoftext|>")[0]

window_size = 3
chunk_size = 4
new_tokens = []
original_tokens = tokenizer.batch_encode_plus(
    text_example,
    padding=True,
)
compressed_tokens = []
replaced_original_tokens_batch = []
# original_tokens
for tokens in original_tokens["input_ids"]:
    original_lines = np.array(tokens)
    pure_tokens = original_lines[original_lines != eos_token_id].tolist()
    print(pure_tokens)
    full_chunks_amount = len(pure_tokens) // (window_size * chunk_size)
    print(full_chunks_amount)
    max_percent = 0.8
    pure_tokens_chunks = list(chunked(pure_tokens, window_size * chunk_size))
    print(pure_tokens_chunks)
    prob = 0.3
    random_mask = np.random.random(int(full_chunks_amount * max_percent))
    mask = random_mask < prob
    chunks_for_tokenization = np.where(mask)[0].tolist()
    chunks_for_tokenization = set(chunks_for_tokenization)
    if len(chunks_for_tokenization) == 0:
        chunks_for_tokenization = set(
            [
                random.randint(
                    0,
                    int(full_chunks_amount * max_percent),
                )
            ]
        )
    print("chunks_for_tokenization ", chunks_for_tokenization)
    replaced_original_tokens = []
    new_input_tokens = []
    for i, tokens in enumerate(pure_tokens_chunks):
        if i in chunks_for_tokenization:
            replaced_original_tokens.extend([text_token_id] * len(tokens))
            new_input_tokens.extend([text_token_id] * chunk_size)
        else:
            replaced_original_tokens.extend(tokens)
            new_input_tokens.extend(tokens)
    print("=== replaced_original_tokens")
    print(tokenizer.decode(replaced_original_tokens))
    print("=== new_input_tokens")
    print(tokenizer.decode(new_input_tokens))
    print(len(new_input_tokens))
    compressed_tokens.append(new_input_tokens)
    replaced_original_tokens_batch.append(replaced_original_tokens)
    print("==")
    print("==")
    print("==")

compressed_tokens_attention = []
max_compressed_len = max([len(item) for item in compressed_tokens])
max_replaced_len = max([len(item) for item in replaced_original_tokens_batch])

for compressed_seq, replaced_seq in zip(
    compressed_tokens,
    replaced_original_tokens_batch,
):
    compressed_seq_len = len(compressed_seq)
    replaced_seq_len = len(replaced_seq)
    attention_mask = [1] * (compressed_seq_len)

    if compressed_seq_len < max_compressed_len:
        compressed_seq += [eos_token_id] * (max_compressed_len - compressed_seq_len)
        attention_mask += [0] * (max_compressed_len - compressed_seq_len)

    if compressed_seq_len < max_replaced_len:
        replaced_seq += [eos_token_id] * (max_replaced_len - replaced_seq_len)

    compressed_tokens_attention.append(attention_mask)
# len(compressed_tokens[0]), len(compressed_tokens[1])
# compressed_tokens = torch.tensor(compressed_tokens)
# compressed_tokens_attention = torch.tensor(compressed_tokens_attention)
np.array(compressed_tokens).shape, np.array(
    replaced_original_tokens_batch
).shape, np.array(original_tokens["input_ids"]).shape

[33464, 6832, 11, 264, 1186, 2566, 315, 5662, 6832, 11, 702, 13791, 1506, 279, 18414, 315, 20443, 11229, 304, 3213, 1635, 13, 5542, 656, 59711, 9331, 311, 34549, 15712, 11, 1181, 8357, 525, 10454, 14756, 70767, 13, 1988, 1128, 6896, 374, 5538, 6832, 30, 1597, 1128, 3643, 432, 773, 7988, 30, 220]
4
[[33464, 6832, 11, 264, 1186, 2566, 315, 5662, 6832, 11, 702, 13791], [1506, 279, 18414, 315, 20443, 11229, 304, 3213, 1635, 13, 5542, 656], [59711, 9331, 311, 34549, 15712, 11, 1181, 8357, 525, 10454, 14756, 70767], [13, 1988, 1128, 6896, 374, 5538, 6832, 30, 1597, 1128, 3643, 432], [773, 7988, 30, 220]]
chunks_for_tokenization  {0}
=== replaced_original_tokens
<|object_ref_start|><|object_ref_start|><|object_ref_start|><|object_ref_start|><|object_ref_start|><|object_ref_start|><|object_ref_start|><|object_ref_start|><|object_ref_start|><|object_ref_start|><|object_ref_start|><|object_ref_start|>ized the landscape of artificial intelligence in recent years. From self-driving cars to persona

((2, 98), (2, 122), (2, 122))

In [42]:
original_tokens_torch = torch.tensor(
    original_tokens["input_ids"],
    device="cuda",
)
replaced_tokens_torch = torch.tensor(
    replaced_original_tokens_batch,
    device="cuda",
)
compressed_tokens_torch = torch.tensor(
    compressed_tokens,
    device="cuda",
)
# torch.Size([2, 51, 896])
original_embeds = model.get_input_embeddings()(original_tokens_torch)
replaced_embeds = model.get_input_embeddings()(replaced_tokens_torch)
# torch.Size([2, 35, 896]) 51 - 3*4*2 + 4*2
compressed_embeds_template = model.get_input_embeddings()(compressed_tokens_torch)
compressed_embeds_template.shape, original_embeds.shape, replaced_embeds.shape

(torch.Size([2, 98, 896]),
 torch.Size([2, 122, 896]),
 torch.Size([2, 122, 896]))

In [32]:
compressed_embeds_template.dtype, original_embeds.dtype, replaced_embeds.dtype

(torch.bfloat16, torch.bfloat16, torch.bfloat16)

In [33]:
text_token_id

151646

In [43]:
tokens_for_compression_mask = replaced_tokens_torch == text_token_id
compressed_tokens_mask = compressed_tokens_torch == text_token_id
# original_tokens_torch[tokens_for_compression_mask].shape
original_embeds[tokens_for_compression_mask].shape, compressed_tokens_torch[
    compressed_tokens_mask
].shape

(torch.Size([48, 896]), torch.Size([16]))

In [44]:
embeds_for_compression = original_embeds[tokens_for_compression_mask].reshape(
    -1,
    window_size,
    original_embeds.shape[-1],
)
embeds_for_compression.shape

torch.Size([16, 3, 896])

In [45]:
pooled_embeds = embed_pooler_v2(embeds_for_compression)
# pooled_embeds = pooled_embeds.reshape(pooled_embeds.shape[0], -1)
pooled_embeds.shape, pooled_embeds.dtype

(torch.Size([16, 1, 896]), torch.bfloat16)

In [38]:
import torch

self = torch.tensor(
    [
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
    ]
)
mask = torch.tensor(
    [
        [0, 0, 0, 0, 1],
        [1, 1, 0, 1, 1],
    ],
    dtype=torch.bool,
)
source = torch.tensor(
    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
)
self.masked_scatter_(mask, source)

tensor([[0, 0, 0, 0, 1],
        [2, 3, 0, 4, 5]])

In [52]:
compressed_embeds_template.shape

torch.Size([2, 98, 896])

In [74]:
compressed_tokens_mask.shape

torch.Size([2, 98])

In [75]:
compressed_embeds_template.shape

torch.Size([2, 98, 896])

In [76]:
compressed_tokens_mask.shape
# .expand_as(compressed_embeds_template).shape

torch.Size([2, 98])

In [ ]:
compressed_tokens_mask.unsqueeze(-1).expand_as(compressed_embeds_template).shape

torch.Size([2, 98, 896])

In [83]:
compressed_embeds_template.dtype, pooled_embeds.dtype

(torch.bfloat16, torch.float32)

In [48]:
compressed_embeds_template.masked_scatter(
    compressed_tokens_mask.unsqueeze(-1).expand_as(compressed_embeds_template),
    pooled_embeds,
).shape

torch.Size([2, 98, 896])

In [50]:
compressed_embeds_template = compressed_embeds_template.masked_scatter_(
    compressed_tokens_mask.unsqueeze(-1).expand_as(compressed_embeds_template),
    pooled_embeds,
)

In [ ]:
compressed_embeds_template[compressed_tokens_mask][0]

In [ ]:
compressed_embeds_template[compressed_tokens_mask][0]

In [53]:
labels = compressed_tokens_torch.clone()
text_token_id = tokenizer.encode("<|object_ref_start|>")[0]
labels[labels == text_token_id] = -100
# labels

In [56]:
model_output_1 = model(
    inputs_embeds=compressed_embeds_template,
    labels=labels,
    output_hidden_states=True,
)
model_output_1.loss

tensor(5.9123, device='cuda:0')